<a href="https://colab.research.google.com/github/Trevorchenmsu/FB-project-code-backup/blob/main/remove_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# notebook setting set as GPU
from google.colab import drive
drive.mount('/content/drive/')
import os
home = "/content"
os.chdir(home)
os.getcwd()

Mounted at /content/drive/


'/content'

In [2]:
# Check how many images we have in total
!ls /content/drive/MyDrive/data/img/ | wc -l

12143


In [3]:
!pip install fire
!pip install easyocr
!pip install torch==1.6.0 torchvision==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

     |████████████████████████████████| 92kB 5.6MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=5898cd8c01224137f05e6e1eeb2246a4e25fceab6418faa8d49ebe056685ed6a
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
Successfully built fire
     |████████████████████████████████| 63.0MB 44kB/s 
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 552.8MB 31kB/s 
     |████████████████████████████████| 5.8MB 55.2MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.6.0+cu92 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101


In [4]:
import os
import glob
import json
import shutil
from multiprocessing import Pool

import fire
import easyocr
import numpy as np
import torch

from PIL import Image
from skimage import transform
from skimage.feature import canny
from skimage.color import rgb2gray, gray2rgb


In [ ]:
# generate OCR json

root_dir = "/content/drive/MyDrive/data/"
ocr_json_dir = os.path.join(root_dir, "ocr.json")

def cast_pred_type(pred):
    result = []
    for tup in pred:
        coord, txt, score = tup
        coord = np.array(coord).tolist()
        score = float(score)
        result.append((coord, txt, score))
    return result

def detect(root_dir):
    reader = easyocr.Reader(['en'])
    image_dir = os.path.join(root_dir, 'img')
    images = glob.glob(os.path.join(image_dir, '*.png'))
    images += glob.glob(os.path.join(image_dir, '**', '*.png'))
    # images = images[:3]
    # assert len(images) > 9000

    out_json = os.path.join(root_dir, 'ocr.json')
    out_anno = {}
    print(f"Find {len(images)} images!")

    for i, image_path in enumerate(images):
        print(F"{i}/{len(images)}")
        img_name = os.path.basename(image_path)
        pred = reader.readtext(image_path)
        out_anno[img_name] = cast_pred_type(pred)

    with open(out_json, 'w') as f:
        json.dump(out_anno, f)

detect(root_dir)

In [ ]:
# generate ORC box json

json_dir = os.path.join(root_dir, "ocr.box.json")

def point_to_box(anno_json):
    with open(anno_json, 'r') as f:
        ocr_anno = json.load(f)
    
    boxed_anno = {}
    for k, v in ocr_anno.items():
        img_ocr_infos = []
        for txt_info in v:
            coord, txt, score = txt_info
            xmin = min([p[0] for p in coord])
            xmax = max([p[0] for p in coord])
            ymin = min([p[1] for p in coord])
            ymax = max([p[1] for p in coord])
            box = [xmin, ymin, xmax, ymax]
            img_ocr_infos.append([box, txt, score])
        boxed_anno[k] = img_ocr_infos
    
    out_path = anno_json.replace('.json', '.box.json')
    with open(out_path, 'w') as f:
        json.dump(boxed_anno, f)
    print("done!")

    point_to_box(ocr_json_dir)  

In [5]:
# generate the mask and the images with removed text

img_dir = os.path.join(root_dir, "img")
out_dir = os.path.join(root_dir, "img_OCR")
# ocr_json_dir = "/content/drive/MyDrive/data/ocr.json"
# json_dir = "/content/drive/MyDrive/data/ocr.box.json"
# img_dir = "/content/drive/MyDrive/data/img"
# out_dir = "/content/drive/MyDrive/data/img_out"

def multi_boxes_mask(image, boxes, pad_crop=5):
    """
    image: np.uint8 (h, w, c)
    boxes: np.int32 (n, 4) ymin, xmin, ymax, xmax
    """
    image = image.copy()
    mask = np.zeros_like(image)
    ih, iw, ic = image.shape
    resize = lambda a, b: transform.resize(a, b, preserve_range=True).astype(np.uint8)
    import matplotlib.pyplot as plt
    
    for box in boxes:
        # image[box[0]: box[2], box[1]: box[3], :] = 0
        box[:2] = np.maximum(box[:2] - pad_crop, 0)
        box[2:] = np.minimum(box[2:] + pad_crop, image.shape[:2])
        
        patch = image[box[0]: box[2], box[1]: box[3], :]
        pure_white = (patch > 253).all(axis=-1).astype(np.uint8)
        mask[box[0]: box[2], box[1]: box[3], :] = pure_white[..., None]
        
        # plt.subplot(2, 1, 1)
        # plt.imshow(patch)
        # plt.subplot(2, 1, 2)
        # plt.imshow(pure_white)
        # plt.colorbar()
        # plt.show()
        
        print('pure_white ', pure_white.sum())
    
    shift = 3
    shifts = [
        (0, 0), (shift, 0), (-shift, 0), (0, shift), (0, -shift),
        (shift, shift), (-shift, shift), (shift, -shift), (-shift, -shift)
    ]
    # shifts = []
    for offset in shifts:
        ox, oy = offset
        _mask = mask.copy()

        slice_y = slice(max(0, 0 + oy), min(ih, ih + oy))
        slice_x = slice(max(0, 0 + ox), min(iw, iw + ox))
        print(slice_y, slice_x)
        _mask = _mask[
            max(0, 0 + oy): min(ih, ih + oy),
            max(0, 0 + ox): min(iw, iw + ox),
            :
        ]
        crop_pad = [
            (max(0, -oy), max(0, oy)),
            (max(0, -ox), max(0, ox)),
            (0, 0)
        ]
        _mask = np.pad(_mask, crop_pad)
        print(
            crop_pad,
            np.abs(_mask - mask).sum(),
            np.abs(mask - np.clip(_mask + mask, 0, 1)).sum()
        )
        mask = np.clip(_mask + mask, 0, 1)

    image = image * (1 - mask) + mask * 255 * 0
    mask *= 255
    return image, mask

def _mask_white_txt(args):
    img_name, img_boxes, img_dir, out_dir = args
    img_path = os.path.join(img_dir, img_name)
    out_path = os.path.join(out_dir, img_name)
    
    if os.path.exists(out_path):
        return
    # if img_name != '01487.png':
    #     continue
    
    print(out_path)
    img_boxes = [box_info[0] for box_info in img_boxes]
    if len(img_boxes) > 0:
        boxes = np.asarray(img_boxes, dtype=np.int32)
        # print(boxes)
        boxes = np.concatenate([boxes[:, ::-1][:, 2:], boxes[:,::-1][:, :2]], axis=1)
        # print(boxes)
        # x,y,x,y -> y,x,y,x
        img = np.array(Image.open(img_path).convert('RGB'))
        # res = inpaint_model.inpaint_multi_boxes(img, boxes)
        masked_img, mask = multi_boxes_mask(img, boxes)

        Image.fromarray(masked_img).save(out_path)
        out_path = os.path.join(out_dir, img_name.replace('.png', '.mask.png'))
        Image.fromarray(mask).save(out_path)
    else:
        img = np.asarray(Image.open(img_path).convert('RGB'))
        shutil.copy(img_path, out_path)

        mask = np.zeros_like(img)
        out_path = os.path.join(out_dir, img_name.replace('.png', '.mask.png'))
        Image.fromarray(mask).save(out_path)

def generate_mask(ocr_box_anno, img_dir, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    with open(ocr_box_anno, 'r') as f:
        boxes_anno = json.load(f)

    # for i, (img_name, img_boxes) in enumerate(boxes_anno.items()):
    #     pass
    
    with Pool(16) as pool:
        args = [
            (img_name, img_boxes, img_dir, out_dir)
            for img_name, img_boxes in boxes_anno.items()
        ]
        pool.map(_mask_white_txt, args)
    print("done!")


generate_mask(json_dir, img_dir, out_dir)

/usr/local/lib/python3.7/dist-packages/torch/cuda/__init__.py:125: UserWarning: 
Tesla T4 with CUDA capability sm_75 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the Tesla T4 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


Streaming output truncated to the last 5000 lines.
slice(0, 366, None) slice(3, 550, None)
slice(0, 306, None) slice(0, 547, None)
[(0, 3), (0, 3), (0, 0)] 2182590 1962225
[(3, 0), (0, 3), (0, 0)] 1048044 973845
pure_white  3976
[(3, 0), (3, 0), (0, 0)] 2881536 2870280
slice(0, 710, None) slice(3, 512, None)
slice(3, 365, None) slice(0, 547, None)
slice(0, 384, None) slice(0, 512, None)
slice(0, 397, None) slice(0, 297, None)
[(0, 0), (0, 3), (0, 0)] 2151936 2143530
[(0, 3), (0, 3), (0, 0)] 3045120 3033225
[(3, 0), (3, 0), (0, 0)] 1088160 669375
[(0, 3), (3, 0), (0, 0)] 2004183 1725840
slice(0, 366, None) slice(0, 547, None)
[(0, 3), (3, 0), (0, 0)] 2133567 970785
slice(0, 362, None) slice(3, 550, None)
[(0, 0), (0, 0), (0, 0)] 0 0
[(0, 0), (0, 3), (0, 0)] 1641984 1635570
slice(3, 549, None) slice(0, 822, None)
[(0, 0), (3, 0), (0, 0)] 1963776 1956105
slice(0, 384, None) slice(3, 512, None)
slice(3, 366, None) slice(0, 550, None)
slice(0, 710, None) slice(0, 509, None)
slice(0, 797, No

In [8]:
# Check how many images we have in total
!ls /content/drive/MyDrive/data/img_OCR/ | wc -l

24286


In [7]:
!zip -r /content/drive/MyDrive/data/img_OCR_2.zip /content/drive/MyDrive/data/img_OCR

Streaming output truncated to the last 5000 lines.
  adding: content/drive/MyDrive/data/img_OCR/26940.mask.png (deflated 40%)
  adding: content/drive/MyDrive/data/img_OCR/52361.png (deflated 0%)
  adding: content/drive/MyDrive/data/img_OCR/30682.mask.png (deflated 48%)
  adding: content/drive/MyDrive/data/img_OCR/20541.png (deflated 0%)
  adding: content/drive/MyDrive/data/img_OCR/71058.mask.png (deflated 30%)
  adding: content/drive/MyDrive/data/img_OCR/94230.png (deflated 0%)
  adding: content/drive/MyDrive/data/img_OCR/70125.png (deflated 0%)
  adding: content/drive/MyDrive/data/img_OCR/68512.mask.png (deflated 50%)
  adding: content/drive/MyDrive/data/img_OCR/96308.png (deflated 0%)
  adding: content/drive/MyDrive/data/img_OCR/93128.mask.png (deflated 41%)
  adding: content/drive/MyDrive/data/img_OCR/97348.png (deflated 0%)
  adding: content/drive/MyDrive/data/img_OCR/91048.mask.png (deflated 38%)
  adding: content/drive/MyDrive/data/img_OCR/48970.png (deflated 0%)
  adding: conten